In [2]:
#import required libraries
import pandas as pd
import numpy as np
import os
#!pip install folium
import folium
from datetime import datetime

In [3]:
ct = datetime.utcnow()
#if(ct.minute == 15):
  #url = f'https://www.airnowapi.org/aq/data/?startDate={ct.year}-{ct.month}-{ct.day-1}T{ct.hour%24}&endDate={ct.year}-{ct.month}-{ct.day}T{ct.hour}&parameters=OZONE,PM25,PM10,CO,NO2,SO2&BBOX=-127.369132,25.826774,-51.783195,49.401810&dataType=A&format=text/csv&verbose=1&monitorType=2&includerawconcentrations=1&API_KEY=211FD3EB-9CFC-47AE-9657-FD48194D0382'
  #data = pd.read_csv(url, header=None)
url = f'https://www.airnowapi.org/aq/data/?startDate={ct.year}-{ct.month}-{ct.day}T{ct.hour}&endDate={ct.year}-{ct.month}-{ct.day}T{ct.hour}&parameters=OZONE,PM25,PM10,CO,NO2,SO2&BBOX=-127.369132,25.826774,-51.783195,49.401810&dataType=A&format=text/csv&verbose=1&monitorType=2&includerawconcentrations=1&API_KEY=211FD3EB-9CFC-47AE-9657-FD48194D0382'
data = pd.read_csv(url, header=None)
#data

In [4]:
data

,0,1,2,3,4,5,6,7,8,9,10
0,49.261014,-124.806628,2020-11-07T00:00,PM2.5,8.0000,26,1,Port Alberni Elementary,British Columbia Ministry of Environment,000102602,124000102602
1,48.286700,-124.620300,2020-11-07T00:00,OZONE,29.1360,28,1,Cheeka Peak,Washington Department of Ecology,530090013,840530090013
2,48.286700,-124.620300,2020-11-07T00:00,PM2.5,1.7000,6,1,Cheeka Peak,Washington Department of Ecology,530090013,840530090013
3,48.286700,-124.620300,2020-11-07T00:00,SO2,0.1000,0,1,Cheeka Peak,Washington Department of Ecology,530090013,840530090013
4,48.286700,-124.620300,2020-11-07T00:00,CO,0.1262,-999,0,Cheeka Peak,Washington Department of Ecology,530090013,840530090013
...,...,...,...,...,...,...,...,...,...,...,...
2963,49.019400,-55.802800,2020-11-07T00:00,OZONE,31.0000,27,1,Grand Falls Windsor,Newfoundland & Labrador DEC,000010501,124000010501
2964,47.098991,-55.198521,2020-11-07T00:00,PM2.5,9.0000,38,1,Marystown/Burin,Environment Canada,000010901,124000010901
2965,47.652800,-52.816700,2020-11-07T00:00,OZONE,34.0000,30,1,St. John's,Newfoundland & Labrador DEC,000010102,124000010102
2966,47.505000,-52.794700,2020-11-07T00:00,OZONE,36.0000,33,1,Mount Pearl,Newfoundland & Labrador DEC,000010401,124000010401


In [5]:
#data.columns = ['Latitude', 'Longitude', 'DateObserved', 'ParameterName', 'Concentration','AQI', 'CategoryNumber', 'ReportingArea', 'Source', 'SiteNumber', 'SiteNumberExtension']
data.columns = ['Latitude', 'Longitude', 'DateObserved', 'ParameterName', 'Concentration','AQI', 'CategoryNumber', 'ReportingArea', 'Source', 'SiteNumber', 'SiteNumberExtension']

data.sort_values('Latitude')

#add category names based on the category number
category_names = []

for i in range(0,len(data)):
  if(data.loc[i,'AQI']<=50):
    category_names.append('Good')
  elif(data.loc[i,'AQI']<=100):
    category_names.append('Moderate')
  elif(data.loc[i,'AQI']<=150):
    category_names.append('Unhealthy for Sensitive Groups')
  elif(data.loc[i,'AQI']<=200):
    category_names.append('Unhealthy')
  elif(data.loc[i,'AQI']<=300):
    category_names.append('Very Unhealthy')
  elif(data.loc[i,'AQI']<=500):
    category_names.append('Hazardous')
  else:
    category_names.append('Remove')

category_names
data.insert(7,'CategoryName',category_names,True)

In [10]:
len(category_names)

2966

In [6]:
#removing irrelevant category numbers
data = data[data['CategoryNumber']!=0]
data = data[data['CategoryName']!='Remove']

In [7]:
#lat_lon = data.loc[0][0:2]
lat_lon = pd.DataFrame()
lat_lon = lat_lon.append(data[data.columns[0:2]])

In [8]:
lat_lon = lat_lon.drop_duplicates()

In [106]:
lat_lon.to_pickle('lat_lon.pkl')

In [9]:
data

,Latitude,Longitude,DateObserved,ParameterName,Concentration,AQI,CategoryNumber,CategoryName,ReportingArea,Source,SiteNumber,SiteNumberExtension
0,49.261014,-124.806628,2020-11-07T00:00,PM2.5,8.000,26,1,Good,Port Alberni Elementary,British Columbia Ministry of Environment,000102602,124000102602
1,48.286700,-124.620300,2020-11-07T00:00,OZONE,29.136,28,1,Good,Cheeka Peak,Washington Department of Ecology,530090013,840530090013
2,48.286700,-124.620300,2020-11-07T00:00,PM2.5,1.700,6,1,Good,Cheeka Peak,Washington Department of Ecology,530090013,840530090013
3,48.286700,-124.620300,2020-11-07T00:00,SO2,0.100,0,1,Good,Cheeka Peak,Washington Department of Ecology,530090013,840530090013
5,48.366058,-124.610046,2020-11-07T00:00,PM2.5,8.200,21,1,Good,Neah Bay 2-Makah Tribe,Washington Department of Ecology,530090015,840530090015
...,...,...,...,...,...,...,...,...,...,...,...,...
2963,49.019400,-55.802800,2020-11-07T00:00,OZONE,31.000,27,1,Good,Grand Falls Windsor,Newfoundland & Labrador DEC,000010501,124000010501
2964,47.098991,-55.198521,2020-11-07T00:00,PM2.5,9.000,38,1,Good,Marystown/Burin,Environment Canada,000010901,124000010901
2965,47.652800,-52.816700,2020-11-07T00:00,OZONE,34.000,30,1,Good,St. John's,Newfoundland & Labrador DEC,000010102,124000010102
2966,47.505000,-52.794700,2020-11-07T00:00,OZONE,36.000,33,1,Good,Mount Pearl,Newfoundland & Labrador DEC,000010401,124000010401


In [10]:
i=1
locations = []
for g in data.groupby(['Latitude','Longitude']):
  #print(i)
  locations.append(g)
  i=i+1

In [11]:
#for getting the info of the max pollutant
max_pollutant_info = []
parameter = "O3"


for loc in range(0,len(locations)):
  max_pollutant_one_location = []
  date_index = np.where(locations[loc][1]['DateObserved'].values==(max(locations[loc][1]['DateObserved'].values)))

  #print(locations[loc][1]['AQI'].values) #printing all AQI values
  index = np.where(locations[loc][1]['AQI'].values[date_index]==(max(locations[loc][1]['AQI'].values[date_index])))
  index = np.array(index+date_index[0][0]).tolist()[0][0]

  #print(index[0]) #checking if the max is chosen
  #print(locations[loc][1]['ParameterName'].values[index], locations[loc][1]['CategoryName'].values[index]) #checking if the required values ar eprinted correctly
  max_pollutant_one_location.append(index)
  max_pollutant_one_location.append(np.array(date_index).tolist()[0][0])
  max_pollutant_one_location.append(locations[loc][1]['DateObserved'].values[index])
  max_pollutant_one_location.append(locations[loc][1]['Latitude'].values[index])
  max_pollutant_one_location.append(locations[loc][1]['Longitude'].values[index])
  max_pollutant_one_location.append(locations[loc][1]['ReportingArea'].values[index])
  max_pollutant_one_location.append(locations[loc][1]['ParameterName'].values[index])
  max_pollutant_one_location.append(locations[loc][1]['AQI'].values[index])
  max_pollutant_one_location.append(locations[loc][1]['CategoryName'].values[index])

  max_pollutant_info.append(max_pollutant_one_location)

In [38]:
#colors for different air quality groups
icon_map = {
    'Good': r"icons/s1.png",
    'Moderate': r"icons/s2.png",
    'Unhealthy for Sensitive Groups': r"icons/s3.png",
    'Unhealthy': r"icons/s3.png",
    'Very Unhealthy': r"icons/s4.png",
    'Hazardous': r"icons/s5.png"
}

color_map = {
    'Good': '#00e400',
    'Moderate': 'ffff00',
    'Unhealthy for Sensitive Groups': '#ff7e00',
    'Unhealthy': '#ff0000',
    'Very Unhealthy': '#8f3f97',
    'Hazardous': '#7e0023'
}

#pollutant type
pollutant = "OZONE"

#radius of the circle in meters (converting from miles to meters)
radius = 700 * 1.60934

In [41]:
' (' + f'<span style="color:{color_map[max_pollutant_info[i][8]]};>' + locations[i][1]['CategoryName'].values.tolist()[j]+ '</span>'

' (<span style="color:#00e400;>Good</span>'

In [45]:
#display St.Louis map in the beginning
lat = 38.580917
lon = -90.244598
#creates and stores the map in m
m = folium.Map(location=[lat, lon], zoom_start=12, tiles="Stamen Terrain", zoom_control=False)
#m = folium.Map(location=[lat, lon], zoom_start=12, zoom_control=False,
  #tiles = 'https://tiles.stadiamaps.com/tiles/osm_bright/{z}/{x}/{y}{r}.png?api_key=446b4cdf-1001-4780-83dd-1dabe1b99d5c',
          #attr = '&copy; <a href="https://stadiamaps.com/">Stadia Maps</a>, &copy; <a href="https://openmaptiles.org/">OpenMapTiles</a> &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors')


for i in range(0,len(max_pollutant_info)):

  popup_text = ''
  for j in range(max_pollutant_info[i][1],len(locations[i][1])):
    popup_text =  popup_text + '<b>' + locations[i][1]['ParameterName'].values.tolist()[j] + '</b>' + ': ' + str(locations[i][1]['AQI'].values.tolist()[j]) + ' (' + '<span style="color: '+ locations[i][1]['CategoryName'].values.tolist()[j] +';>' + locations[i][1]['CategoryName'].values.tolist()[j]+ '</span>' + ')'
    popup_text = popup_text + '<br>'

  iframe = folium.IFrame(popup_text, figsize=(3,0.6+(0.32*(len(locations[i][1])-max_pollutant_info[i][1]))))
  popup = folium.Popup(iframe)

  icon_path = icon_map[max_pollutant_info[i][8]]

  icon = folium.features.CustomIcon(icon_image=icon_path ,icon_size=(30,52),icon_anchor=(15,30))

  marker=folium.Marker([max_pollutant_info[i][3], max_pollutant_info[i][4]],
                       icon=icon,
                       tooltip='Dominant Pollutant: {}'.format(max_pollutant_info[i][6]),
                       popup=popup).add_to(m)



In [46]:
m.save('map.html')

In [58]:
ct

datetime.datetime(2020, 12, 1, 0, 0)

In [59]:
dt = ct

In [64]:
ct.date()

datetime.date(2020, 12, 1)

In [65]:
dt.date()

datetime.date(2020, 12, 1)

In [69]:
ct.date() == dt.date()

False